In [37]:
import pandas as pd
trading_date = '01/01/2019'

price_fields = ['AVERAGE PRICE', 'WEIGHTED AVG. PRICE', 'AVG. PRICE BOTTOM 5 TRADES', '2ND QUARTILE PRICE',
                '3RD QUARTILE PRICE', '4TH QUARTILE PRICE', 'AVG. PRICE TOP 5 TRADES', 'STANDARD DEVIATION',
                'VOLUME OF TRADES (000\'S)', 'NUMBER OF TRADES']
    
df = pd.read_excel('FINRA_IDS_PXTABLES-20190102.xlsx', sheet_name='MBS', skiprows=8, header=None)
df[0] = df[1].where(df[1].isin(price_fields)).fillna(method='ffill')
df[1] = df[1].fillna('0')

#slicing off bottom again
df = df.loc[:df[df[1].str.contains('Indicates')].index.values[0] - 2]
df[9] = df[1].where(df[0].isna()).fillna(method='ffill')
df

,0,1,2,3,4,5,6,7,8,9
0,NaN,PRICING TABLE: AGENCY PASS-THRU (SPECIFIED) - ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRICING TABLE: AGENCY PASS-THRU (SPECIFIED) - ...
1,NaN,Asset Sub-Class / Metric,COUPON,NaN,NaN,NaN,NaN,NaN,NaN,Asset Sub-Class / Metric
2,NaN,FNMA,<= 2.5,3,3.5,4,4.5,5,> 5.0,FNMA
3,AVERAGE PRICE,AVERAGE PRICE,97.9,99.2,100.2,101.2,99.2,*,100.4,FNMA
4,WEIGHTED AVG. PRICE,WEIGHTED AVG. PRICE,98.2,99.9,99.9,101.2,104.8,*,107.8,FNMA
5,AVG. PRICE BOTTOM 5 TRADES,AVG. PRICE BOTTOM 5 TRADES,96.6,97,97.9,99.5,97.2,*,100.4,FNMA
6,2ND QUARTILE PRICE,2ND QUARTILE PRICE,97.7,98.6,99.7,99.4,92,*,*,FNMA
7,3RD QUARTILE PRICE,3RD QUARTILE PRICE,98.6,99.7,100.7,101.2,103.4,*,*,FNMA
8,4TH QUARTILE PRICE,4TH QUARTILE PRICE,98.8,100.1,101,102.5,104.5,*,*,FNMA
9,AVG. PRICE TOP 5 TRADES,AVG. PRICE TOP 5 TRADES,98.8,100.3,101.5,104.1,103.9,*,100.4,FNMA


In [111]:
import pandas as pd
import numpy as np
import os

trading_date = '01/01/2019'

price_fields = ['AVERAGE PRICE', 'WEIGHTED AVG. PRICE', 'AVG. PRICE BOTTOM 5 TRADES', '2ND QUARTILE PRICE',
                '3RD QUARTILE PRICE', '4TH QUARTILE PRICE', 'AVG. PRICE TOP 5 TRADES', 'STANDARD DEVIATION',
                'VOLUME OF TRADES (000\'S)', 'NUMBER OF TRADES']
price_fields_subcategory = ['CUSTOMER BUY', 'CUSTOMER SELL', 'DEALER TO DEALER', '<= $1MM', '<= $10MM', 
                            '<= $100MM', '> $100MM']
    
df = pd.read_excel('FINRA_IDS_PXTABLES-20190102.xlsx', sheet_name='AgencyCMO', skiprows=8, header=None)
df.replace(np.NaN, '', inplace=True)
df = df.loc[:df[df[1].str.contains('Indicates')].index.values[0] - 2]
df[0] = df[1].where(df[1].isin(price_fields)).fillna(method='ffill')

df[6] = df[1].where(df[1].isin(price_fields_subcategory))
df[7] = df[1].where(~((df[1].isin(price_fields) ^ (df[1].isin(price_fields_subcategory))))).fillna(method='ffill')
df[8] = df[1].where(df[1].str.contains('PRICING TABLE')).fillna(method='ffill')
df = df[~(df[1] == df[7])]
df.drop(1, axis=1, inplace=True)
df[8] = df[8].str.replace('PRICING TABLE: ', '')
df[8] = df[8].str.replace(' - BY DEAL VINTAGE', '')
df.columns = ['Measure', 'Pre-2009', '2009-2013', '2014-2016', 'Post-2016', 'Measure2', 'Agency', 'MortgageType']
df

df.Measure2 = df.Measure2.fillna('TOTAL')
df['Date'] = trading_date

# check to see if filepath exists and append headers if they don't
if os.path.isfile('agencycmoprices.csv'):
    pass
else:
    pd.DataFrame(df.columns).transpose().to_csv('agencycmoprices.csv', header=False, index=False, mode='a')

df.to_csv('agencycmoprices.csv', header=False, index=False, mode='a')

In [115]:
import zipfile
import os
import pandas as pd
directory = os.fsencode('data')

for zfile in os.listdir(directory):
        filename = os.fsdecode(zfile)
        zf = zipfile.ZipFile('data/' + filename)

        month_list = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                      'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

        price_fields = ['AVERAGE PRICE', 'WEIGHTED AVG. PRICE', 'AVG. PRICE BOTTOM 5 TRADES', '2ND QUARTILE PRICE',
                        '3RD QUARTILE PRICE', '4TH QUARTILE PRICE', 'AVG. PRICE TOP 5 TRADES', 'STANDARD DEVIATION',
                        'VOLUME OF TRADES (000\'S)', 'NUMBER OF TRADES']

        price_fields_subcategory = ['CUSTOMER BUY', 'CUSTOMER SELL', 'DEALER TO DEALER', '<= $1MM', '<= $10MM', 
                            '<= $100MM', '> $100MM']
        
        # build namelist of specific files I need
        pxtables_list = [i for i in zf.namelist() if 'PXTABLES' in i]

        for trading_day in pxtables_list:
            # get trading day date
            trading_date = trading_day.split('-')[1].replace('.xlsx', '')

            # should i fix trading date?
            trading_date = '{}/{}/{}'.format(trading_date[4:6], trading_date[6:], trading_date[:4])

            # build monthlist:
            current_month = int(trading_date[0:2])
            chart_month_list = {current_month: 'Current Month', (current_month + 1) % 12: 'Current Month + 1',
                                (current_month + 2) % 12: 'Current Month + 2',
                                (current_month + 3) % 12: 'Current Month + 3'}

            # start reading stuff in
            df = pd.read_excel(zf.open(trading_day), sheet_name='AgencyCMO', skiprows=8, header=None)
            
            df.replace(np.NaN, '', inplace=True)
            df = df.loc[:df[df[1].str.contains('Indicates')].index.values[0] - 2]
            df[0] = df[1].where(df[1].isin(price_fields)).fillna(method='ffill')

            df[6] = df[1].where(df[1].isin(price_fields_subcategory))
            df[7] = df[1].where(~((df[1].isin(price_fields) ^ (df[1].isin(price_fields_subcategory))))).fillna(method='ffill')
            df[8] = df[1].where(df[1].str.contains('PRICING TABLE')).fillna(method='ffill')
            df = df[~(df[1] == df[7])]
            df.drop(1, axis=1, inplace=True)
            df[8] = df[8].str.replace('PRICING TABLE: ', '')
            df[8] = df[8].str.replace(' - BY DEAL VINTAGE', '')
            df.columns = ['Measure', 'Pre-2009', '2009-2013', '2014-2016', 'Post-2016', 'Measure2', 'Agency', 'MortgageType']
            df

            df.Measure2 = df.Measure2.fillna('TOTAL')
            df['Date'] = trading_date

            # check to see if filepath exists and append headers if they don't
            if os.path.isfile('agencycmoprices.csv'):
                pass
            else:
                pd.DataFrame(df.columns).transpose().to_csv('agencycmoprices.csv', header=False, index=False, mode='a')

            df.to_csv('agencycmoprices.csv', header=False, index=False, mode='a')